In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [11]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [3]:
'''
    Create Trips from the GPS Data
'''

df.columns

Index(['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.', 'Rego',
       'Latitude', 'Longitude', 'Direction', 'Unnamed: 10', 'Speed', 'RawGPS',
       'Status', 'Flags', 'User Defined', 'Input', 'Output', 'Light', 'Button',
       'Max', 'Average', 'Spd Acc', 'Samples', 'Map Ref', 'Suburb', 'Distance',
       'Position', 'Eng Cool Temp', 'Eng Oil Temp', 'OilPressure',
       'ECM Economy', 'Trip', 'Total Used', 'RPM', 'Tot Hour', 'Brake Hits',
       'Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ', 'Position.1', '  1  ',
       '  2', '  3  ', '  4  ', '  5  ', '  6  ', '  7  ', '  8  ', '  9  ',
       '  10  ', 'Volts', 'Seconds', 'Speed.1', 'Zone 1', 'Zone 2', 'Zone 3',
       'Zone 4', 'Zone 1.1', 'Zone 2.1', 'Zone 3.1', 'Zone 4.1', '  1  .1',
       '  2  ', '  3  .1', '  4  .1', '  5  .1', 'Vehicle', 'Auxiliary',
       '  1  .2', '  2  .1', '  3  .2', '  4  .2', '  5  .2', 'Mass', 'Limit',
       'HDOP', 'Satellites', 'Temperature', 'From Flash', 'Offset Time',
       'Offset'

In [ ]:
'''
    Data Processing:

    Alpha: Check if Odometer Offset exists - Current logic searches for text Odometer Offset in Suburb. Best is to look in 'Offset' Column and get unique values
        Y: process add previous rows odo reading, add offset -- STATUS 1 ***OR***
        Y: filter the rows from the next day -- STATUS 2
        N: proceed usually. 
    1. Filter required columns out of overall dataset
    2. Concat Lat+Long into coords for each row
'''
def manipulate_Odometer_Offsets(df,status):

    odometer_offsets = df[df['Reason'].str.contains("Odometer Offset")].index.tolist()
    
    # Basic Checks
    # print(odometer_offsets)
    if len(odometer_offsets) != 0 or status == 3:
        return df
    if status == 1:
        # Write logic for taking previous rows and add the offset value
        start = 0
        for offset in range(len(odometer_offsets)):
            end = odometer_offsets[offset]
            value = df.loc[end]['Odometer']-df.loc[end-1]['Odometer']
            df.loc[start:end]['Odometer'] += value
            start = end
        return df
    elif status == 2:
        date = None
        for offset in range(len(odometer_offsets)):
            date = pd.to_datetime(df.loc[odometer_offsets[offset]]['GPS']).date()
            date = date + timedelta(days = 1)
        return df[df['GPS'].dt.date >= date]
    
# Donot run odometet manipulator right now!
# df1 =manipulate_Odometer_Offsets(df,2)
def all_preprocessors(df):
    df['Date'] = pd.to_datetime(df['GPS'],format="%Y-%m-%d").dt.date
    df = df[['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.','Speed', 'Rego','Latitude', 'Longitude', 'Direction','Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ','Offset Time','Offset']]
    df.loc[:, 'coords'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)
    return df
    # return manipulate_Odometer_Offsets(df,2)

df = all_preprocessors(df)
# df = manipulate_Odometer_Offsets(df,1)

In [ ]:
'''
    Read and process a list of dates for calendar file.
 
'''

def create_calendar_file(df):
    dates = df['GPS'].dt.date.unique()

    calendardates = [
        {'date': item, 'day_of_week': item.strftime('%A')} for item in dates
    ]
    calendar_cols = ['service_id','start_date','end_date','monday','tuesday','wednesday','thursday','friday','saturday','sunday']
    calendar = pd.DataFrame(columns=calendar_cols)

    for date in calendardates:
        day = date['day_of_week']
        idate = date['date'].strftime('%Y%m%d')
        if  day == 'Monday':
            data = pd.DataFrame([[idate,idate,idate,'1','0','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data],ignore_index=True)
        elif day == 'Tuesday':
            data = pd.DataFrame([[idate,idate,idate,'0','1','0','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Wednesday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','1','0','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Thursday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','1','0','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Friday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','1','0','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Saturday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','1','0']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)
        elif day == 'Sunday':
            data = pd.DataFrame([[idate,idate,idate,'0','0','0','0','0','0','1']],columns = calendar_cols)
            calendar = pd.concat([calendar,data], ignore_index=True)

    if len(calendar) != len(dates):
        print('Some dates have not been exported to Calendar')
    else:
        return calendar
    

# TEST THE FUNCTION HERE
# print(create_calendar_file(df))


In [47]:
'''

    Create Routes from Trips
    
    1. Get all Ignition On and Off events
    2. Get all Logon and Logoff events
    3. Get all Arr and Dep Way Point events

'''

session_starts = []
session_ends = []

current_session_start = None
for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        session_starts.append(current_session_start)
        session_ends.append(index)
        current_session_start = None

session_dataframes = []

# Iterate through the session start and end indices to extract sessions
for start, end in zip(session_starts, session_ends):
    session_df = df.loc[start:end]
    
    # print(session_df.iloc[0]['Odometer']) # - session_df.iloc[end]['Odometer'] > 0)
    # session_dataframes.append(session_df)

# filtered_df = df.loc[df['Reason'].isin(['Ignition On','Ignition Off'])]

223018
223019
223019
223019
223019
223019
223019
223019
223019
223019
223019
223019
223019
223019
223038
223043
223046
223047
223063
223068
223068
223068
223072
223072
223072
223091
223094
223097
223118
223118
223119
223119
223119
223142
223142
223145
223149
223163
223174
223175
223175
223175
223207
223207
223209
223241
223242
223242
223261
223263
223268
223286
223286
223286
223287
223288
223288
223307
223307
223309
223311
223312
223327
223327
223328
223328
223351
223351
223352
223353
223357
223371
223376
223377
223378
223378
223396
223396
223396
223416
223416
223416
223416
223438
223440
223443
223445
223463
223463
223463
223463
223482
223483
223502
223503
223503
223537
223564
223582
223582
223586
223590
223591
223591
223591
223610
223614
223626
223636
223636
223653
223656
223674
223675
223696
223699
223701
223719
223720
223720
223720
223739
223751
223766
223767
223767
223767
223767
223767
223767
223767
223768
223787
223803
0
223829
223831
223840
223874
223874
223876
223923
223924
2239

In [ ]:
# RUN EVERYTHING FROM HERE
